In [1]:
# Importing useful libraries
import requests
from bs4 import BeautifulSoup
import time
import os
import asyncio
import aiohttp
import pandas as pd
import asyncio
from aiohttp import ClientSession, ClientResponseError
from parser import *
from crawler import *
pd.set_option("display.max_colwidth", None)
from functions import *
from engine import *
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ricca\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ricca\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1.1. Get the list of master's degree courses

We start with the list of courses to include in your corpus of documents. In particular, we focus on web scrapping the MSc Degrees. Next, we want you to collect the URL associated with each site in the list from the previously collected list. The list is long and split into many pages. Therefore, we ask you to retrieve only the URLs of the places listed in the first 400 pages (each page has 15 courses, so you will end up with 6000 unique master's degree URLs).

The output of this step is a .txt file whose single line corresponds to the master's URL.

---
Let's use BeautifulSoup and Requests to scrape the links related to universities present on the first 400 pages of the following website:
'https://www.findamasters.com/masters-degrees/msc-degrees'.

Note that it's possible to navigate to different pages by modifying the link in the final part, appending "/?PG=" + the page number.

We are saving the URLs in each line of txt file called 'course_links'. In this case there aren't exceptions to handle with.

In [2]:
# let's create a file .txt called course_links which contains in every line the URL of the link
path = 'course_links.txt'
parser(path)

'The file already exists.'

## 1.2. Crawl master's degree pages
Once you get all the URLs in the first 400 pages of the list, you:

1. Download the HTML corresponding to each of the collected URLs.
2. After you collect a single page, immediately save its HTML in a file. In this way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
3. Organize the downloaded HTML pages into folders. Each folder will contain the HTML of the courses on page 1, page 2, ... of the list of master's programs.

**Tip**: Due to the large number of pages you should download, you can use some methods that can help you shorten the time. If you employed a particular process or approach, kindly describe it.



---
The proposed solution for this task involves creating 400 folders, with each folder dedicated to a web page that was scraped in the previous exercise. Inside each of these folders, the corresponding HTML contents of the 15 website will be stored.

The script is organized into three asynchronous functions:
- *get_info(url, session, folder, page_number)*, which performs an asynchronous HTTP GET request to a line (URL) of the txt file created in the exercise above. It returns the html page, written in a specific folder. It manages exceptions such as "Error 429: Too many requests to the website" and if it happens there's a time.sleep of 1 second, until a new get request is sent.
- *process_batch(urls_session, folder)*, which takes in input 15 urls and creates a list of asynchronous tasks, each corresponding to fetching HTML content from a URL in the given list using the *get_info* function, defined before. It uses asyncio.gather to concurrently execute all tasks and returns the results.
- *main(urls, batch_size, starting_folder)* which creates the path where all the html are put. It iterates through batches of URLs, creating a sub-folder for each batch and calling process_batch to asynchronously download and save HTML content for each URL in the batch.


The Python script is designed for asynchronous tasks using the aiohttp library to fetch HTML content from a list of URLs concurrently. 

In this particular case, working on multiple downloads at the same time are not so effective because the 6000 URLs are all from the same server, so we need to insert a time.sleep to handle "Too many requests". As a consequence, the code takes several hours to complete. To mitigate this, we introduced the *starting_folder* parameter in the main function. This allows us to resume the download process from where it left off, avoiding the need to recreate files and folders from scratch each time.

In [3]:
if __name__ == "__main__":
    if not os.path.exists(path):
        with open(path, 'r') as file:
            urls = [line.strip() for line in file] # creating a list with the 6000 URLs from the lines of course_links.txt
            
        result = await main(urls, starting_folder = 1) # starting_folder represents the folder we are starting from

        for text in result:
            pass # text contains your html (text) response
        print("Download and organization of HTML pages completed.")
    else:
        print('The files already exist.')

The files already exist.


## 1.3 Parse downloaded pages
At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format is as follows:

- Course Name (to save as courseName): string;
- University (to save as universityName): string;
- Faculty (to save as facultyName): string
- Full or Part Time (to save as isItFullTime): string;
- Short Description (to save as description): string;
- Start Date (to save as startDate): string;
- Fees (to save as fees): string;
- Modality (to save as modality):string;
- Duration (to save as duration):string;
- City (to save as city): string;
- Country (to save as country): string;
- Presence or online modality (to save as administration): string;
- Link to the page (to save as url): string.

For each master's degree, you create a course_i.tsv file of this structure:

        courseName \t universityName \t  ... \t url
If an information is missing, you just leave it as an empty string.

---
First things first let's create the empty dataframe with the variables described above.

In [7]:
# Name dataframe columns

columns = [
    "courseName",
    "universityName",
    "facultyName",
    "isItFullTime",
    "description",
    "startDate",
    "fees",
    "modality",
    "duration",
    "city",
    "country",
    "administration",
    "url"
]

# Create a dataframe with the specific columns above
df = pd.DataFrame(columns=columns)

# Visualizza il DataFrame
df


,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url


Now we need to populate the dataframe, opening every html page from every folder created in the exercise 1.2.

After this operation we have to find specific elements in the html page, handling exceptions, for example if there's no matching element with the *find_all* function, variable will be "".

For every row of the dataframe we are also creating a .tsv file named as the master degree, containing the corresponding variables values.

In [8]:
pth = "master_programs_html" # path of the html pages
path = 'course_links.txt'   # URLs path
fold = 0  # change this parameter if you need to start the population process of the df not from folder 1, without losing progresses
if not os.path.exists('files_tsv/course6000.tsv'):
    df = populate_df(pth, path, fold, df)
else:
    print('Already satisfied. There are 21 corrupted links.')

Processing:   0%|          | 0/400 [00:00<?, ?it/s]

Processing: 100%|██████████| 400/400 [1:04:20<00:00,  9.65s/it]

There are 21 links corrupted.


Now let's check if we have considered all the 6000 rows.

In [9]:
df.shape[0]

6000

And print the first and the last 10 rows of the dataframe created.

In [13]:
df.head(5)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,"3D visualisation and animation play a role in many areas, and the popularity of these media just keeps growing. Digital animation provides the eye-catching special effects in the 21st century's favourite films and television shows; 3D design is also essential to everyday work in everything from computer games development, online virtual world development and industrial design to marketing, product design and architecture.GCU's programme in 3D Design for Virtual Environments will help you develop the skills to thrive in a successful career as a visual designer. The programme is practical and career-focused, oriented towards current industry needs, technology and practice. No prior knowledge of 3D design is required.",September,Please see the university website for further information on fees for this course.,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/3d-design-for-virtual-environments-msc/?i93d2645c19223
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,"Businesses and governments rely on sound financial knowledge to underpin their strategies for success.This course will kick-start your professional career, providing you with advanced knowledge of modern financial theories, accounting practices and controls, as well as an understanding of the organisational context.You'll cover fundamental topics such as corporate finance and corporate reporting. Optional modules will allow you specialise your knowledge, ranging from international business finance to forensic accounting. You’ll also be able to apply your learning to practical case studies and simulations to test your knowledge in genuine business scenarios.",September,"UK: £18,000 (Total)International: £34,750 (Total)",MSc,1 year full time,Leeds,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/accounting-and-finance-msc/?i321d3232c3891
2,Accounting and Finance (MSc),University of Bath,School of Management,Full time,"Develop in-depth knowledge of accounting and finance theory and learn to apply it in practice. Graduate prepared for a career in either field.Receive a theoretical foundation in accounting and finance. You’ll learn to analyse accounting information and financial decisions, and how to apply this knowledge in the real world. Our emphasis on technical and practical skills means you will graduate prepared for a finance or accounting career.You'll work with and learn from industry experts. This makes sure you'll explore the most topical issues in finance practice. You'll also develop technical skills employers look for like:",September,Please see the university website for further information on fees for this course.,MSc,1 year full-time,Bath,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/accounting-and-finance-msc/?i280d4409c4025
3,"Accounting, Accountability & Financial Management MSc",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Management MSc course will provide you with an advanced understanding of the economic, technical and institutional foundations of accounting and financial management.You'll explore the importance of accountability and information, corporate governance, and ethics in global financial markets. With an undergraduate degree in a relevant social science subject, you'll be ready to tackle the intellectually challenging world of finance.Through a combination of core modules and your own choice of management areas, you'll come away fully equipped to thrive in any industry you choose to go into.",September,Please see the university website for further information on fees for this course.,MSc,1 year FT,London,

In [12]:
df.tail(5)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
5995,Masters's in Digital Politics and Governance,European School of Political and Social Sciences (ESPOL),Masters Programs,Full time,"Digitalisation is a critical issue in today’s global and European politics. The fast spread of the internet to 63% of global population today and the increased use of digital tools for politics and policy-making bring about new opportunities and challenges. It was hoped that the internet and digital tools would reduce barriers leading to more collective, democratic and borderless collaboration inducing positive social and political change. More recently, the internet is seen to more likely reinforce existing barriers and even as a rising threat to individuals, society, democratic processes and governments. Thus, this master brings together students from different backgrounds to study various political, legal and social problems of today’s digitalisation processes and explore solutions for these new challenges.",See Course,Please see the university website for further information on fees for this course.,MSc,2 years,Lille,France,On Campus,www.findamasters.com/masters-degrees/course/masters-s-in-digital-politics-and-governance/?i3056d8126c69365
5996,Material Culture & Artefact Studies - MSc/PgDip,University of Glasgow,College of Arts & Humanities,Full time&Part time,"Material culture and artefact studies combines the archaeological recovery and specialist examination of an object with its presentation, management and understanding within a cultural",September,Please see the university website for further information on fees for this course.,"MSc,PGDip","9-12 months full-time, 18-24 months part-time",Glasgow,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/material-culture-and-artefact-studies-msc-pgdip/?i307d4953c54108
5997,Material Culture and Gàidhealtachd History MSc,University of the Highlands and Islands,"Arts, Humanities and Business",Part time,"This ground-breaking, internationally acclaimed masters programme offers you a unique opportunity to investigate Gaelic culture – entirely through the language of the Gaels themselves.The course gives you the chance to study in-depth the extraordinary transformations affecting the Gaelic world during recent centuries up to the present day, drawing upon the wealth of material culture, literature, tradition, and language of the people of the Highlands.",September,Please see the university website for further information on fees for this course.,"PGDip,PGCert,MSc",3-6 Years Part-time,Inverness,United Kingdom,Online,www.findamasters.com/masters-degrees/course/material-culture-and-g-idhealtachd-history-msc/?i255d4447c35310
5998,Materials and Manufacturing,Jonkoping University,Masters Programmes,Full time,"Numerical methods and knowledge about the relationship between the manufacturing process, microstructure, and material properties, will make it possible for you to perform component",See Course,Please see the university website for further information on fees for this course.,MSc,2 Years Full Time,Jonkoping,Sweden,On Campus,www.findamasters.com/masters-degrees/course/materials-and-manufacturing/?i2226d8581c64184
5999,Materials and Molecular Modelling MSc,University College London,Department of Chemistry,Full time,"Register your interest in graduate study at UCLThere is a growing need by industry for staff trained in computational molecular and materials sciences. This new multidisciplinary MSc will teach simulation tools used in a wide range of applications, including catalysis and energy materials, nanotechnology and drug design, and will provide transferable skills to other fields, thereby broadening employment prospects.",September,"Full time - £14,100",MSc,1 year full time,London,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/materials-and-molecular-modelling-msc/?i274d1846c70665


# 2. Search Engine


Importing my engine.py file

Note: instead of mentioning how each function, which has been imported from engine.py, works, engine.py is well commented and easy to read! Kindly check it out in parallel. 

In [4]:
from engine import *

First, I will load all the 6000 tsv files into one final dataset.

In [5]:
directory = "./files_tsv"
output_file = "./unique_tsv_file.tsv"
dataset = create_store_tsv(directory, output_file)

Now, I will make a copy to protect the original dataset

In [6]:
dataset.head(5)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,"3D visualisation and animation play a role in many areas, and the popularity of these media just keeps growing. Digital animation provides the eye-catching special effects in the 21st century's favourite films and television shows; 3D design is also essential to everyday work in everything from computer games development, online virtual world development and industrial design to marketing, product design and architecture.GCU's programme in 3D Design for Virtual Environments will help you develop the skills to thrive in a successful career as a visual designer. The programme is practical and career-focused, oriented towards current industry needs, technology and practice. No prior knowledge of 3D design is required.",September,Please see the university website for further information on fees for this course.,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/3d-design-for-virtual-environments-msc/?i93d2645c19223
1,"Agricultural, Environmental and Resource Economics - MSc (Agriculture and Forestry)",University of Helsinki,International Masters Degree Programmes,Full time,"Goal of the pro­grammeAre you looking forward to a future as an expert in Agricultural, Environmental and Resource Economics? As a graduate of our Master’s Programme in Agricultural, Environmental and Resource Economics, you can find employment in the national or international market, for example at universities, research institutes, the public sector or in business, or you can become a self-employed entrepreneur.The Viikki Campus offers optimal resources for studying the unique range of subjects offered by our programme. Upon graduating you will be a professional in applied economics in agricultural, environmental and resource-focused fields. You will be well versed in topics such as climate policy, sustainable agriculture and food security.",September,Tuition fee per year (non-EU/EEA students): 15000 €,MSc,2 years,Helsinki,Finland,On Campus,www.findamasters.com/masters-degrees/course/agricultural-environmental-and-resource-economics-msc-agriculture-and-forestry/?i312d6223c45168
2,Energy Policy and Finance (MSc),University of St Andrews,Interdisciplinary Studies,Full time&Part time,"The MSc in Energy Ethics explores how we balance our energy demands with our concerns about anthropogenic climate change. How can we make sense of the complexity of our energy systems and address challenges of energy security and equity? What can we do to create a better energy future for us all?The challenge of creating a better energy future for us all is both urgent and complex. The MSc in Energy Ethics has been specifically designed to match and address this complexity. It brings together academic experts from across arts, humanities, social and natural sciences to offer an in-depth, interdisciplinary understanding of global, national and local energy challenges.",September,"Home£13,470Overseas£27,230",MSc,"1 year full time, 2 year part time",St Andrews,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/energy-policy-and-finance-msc/?i350d7295c70061
3,Aircraft Design option - Aerospace Vehicle Design (MSc),Cranfield University,Aerospace,Full time,"Study at Cranfield UniversityTo design modern efficient aircraft requires a complex combination of aerodynamic performance, lightweight durable structures and advanced systems engineering. This specialistMSc Aerospace Vehicle Designoption explores how different structural and systems elements can be designed and integrated using up-to-date methods and techniques.Who is it for?This option is suitable for those students wishing to gain an overview of the whole aircraft design process as well as the design of aircraft str

Before going on, to protect the original dataset, let's create a copy of it, deleting the 21 links corrupted that gives us empty values for the variables. Furthermore, as we will be working with the description column, let's replace description missing values with `NA`.

In [7]:
df = dataset.copy()
print(df.shape)
df.description.isnull().sum() # outputs 21
df['description'].fillna("NA", inplace=True)
df = df.dropna(subset=['courseName'], axis=0)
print(df.shape)

(6000, 13)
(5979, 13)


## 2.0.0 Preprocessing the text
First, you must pre-process all the information collected for each MSc by:

  * Removing stopwords
  * Removing punctuation
  * Stemming
  * Anything else you think it's needed
---

Now, as the dataset is loaded, I will apply some preprocessing to all columns. This pre-processing includes, tokenizing, standardazing the text (lower case and removing stop words), and stemming the text of all columns.

In [8]:
desired_columns = [x for x in df.columns if x != "fees"]
for column in desired_columns:
    df[column+"_clean"] = df[column].apply(lambda x: preprocessing(x))

In [9]:
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,...,facultyName_clean,isItFullTime_clean,description_clean,startDate_clean,modality_clean,duration_clean,city_clean,country_clean,administration_clean,url_clean
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,"3D visualisation and animation play a role in many areas, and the popularity of these media just keeps growing. Digital animation provides the eye-catching special effects in the 21st century's favourite films and television shows; 3D design is also essential to everyday work in everything from computer games development, online virtual world development and industrial design to marketing, product design and architecture.GCU's programme in 3D Design for Virtual Environments will help you develop the skills to thrive in a successful career as a visual designer. The programme is practical and career-focused, oriented towards current industry needs, technology and practice. No prior knowledge of 3D design is required.",September,Please see the university website for further information on fees for this course.,MSc,1 year full-time,Glasgow,...,"[school, engin, built, environ]","[full, time]","[visualis, anim, play, role, mani, area, popular, media, keep, grow, digit, anim, provid, eye, catch, special, effect, favourit, film, televis, show, design, also, essenti, everyday, work, everyth, comput, game, develop, onlin, virtual, world, develop, industri, design, market, product, design, architectur, programm, design, virtual, environ, help, develop, skill, thrive, success, career, visual, design, programm, practic, career, focus, orient, toward, current, industri, need, technolog, practic, prior, knowledg, design, requir]",[septemb],[msc],"[year, full, time]",[glasgow],"[unit, kingdom]",[campu],"[www, findamast, com, master, degre, cours, design, virtual, environ, msc]"
1,"Agricultural, Environmental and Resource Economics - MSc (Agriculture and Forestry)",University of Helsinki,International Masters Degree Programmes,Full time,"Goal of the pro­grammeAre you looking forward to a future as an expert in Agricultural, Environmental and Resource Economics? As a graduate of our Master’s Programme in Agricultural, Environmental and Resource Economics, you can find employment in the national or international market, for example at universities, research institutes, the public sector or in business, or you can become a self-employed entrepreneur.The Viikki Campus offers optimal resources for studying the unique range of subjects offered by our programme. Upon graduating you will be a professional in applied economics in agricultural, environmental and resource-focused fields. You will be well versed in topics such as climate policy, sustainable agriculture and food security.",September,Tuition fee per year (non-EU/EEA students): 15000 €,MSc,2 years,Helsinki,...,"[intern, master, degre, programm]","[full, time]","[goal, pro, grammear, look, forward, futur, expert, agricultur, environment, resourc, econom, graduat, master, programm, agricultur, environment, resourc, econom, find, employ, nation, intern, market, exampl, univers, research, institut, public, sector, busi, becom, self, employ, entrepreneur, viikki, campu, offer, optim, resourc, studi, uniqu, rang, subject, offer, programm, upon, graduat, profession, appli, econom, agricultur, environment, resourc, focus, field, well, vers, topic, climat, polici, sustain, agricultur, food, secur]",[septemb],[msc],[year],[helsinki],[finland],[campu],"[www, findamast, com, master, degre, cours, agricultur, environment, resourc, econom, msc, agricultur, forestri]"
2,Energy Policy and Finance (MSc),University of St Andrews,Interdisciplinary Studies,Full time&Part time,"The MSc in Energy Ethics explores how we balance our energy demands with our concerns about anthropogenic climate change. How can we make sense of the co

### 2.0.1 Preprocessing the fees column
Moreover, we want the field `fees` to collect numeric information. As you will see, you scraped textual information for this attribute in the dataset: sketch whatever method you need (using regex, for example, to find currency symbol) to collect information and, in case of multiple information, retrieve only the highest fees. Finally, once you have collected numerical information, you likely will have different currencies: this can be chaotic, so let chatGPT guide you in the choice and deployment of an API to convert this column to a common currency of your choice (it can be USD, EUR or whatever you want). Ultimately, you will have a `float` column renamed `fees (CHOSEN COMMON CURRENCY)`.

-----

Now, it is time to do some preprocessing with the fees column. We have to ensure that all the fees are in Euro (decided) and the column is of dtype float. So, let's do it!

First, I will replace all the entries that are prompting to contact the university or visit webpage. Then, I will see what currency symbols have been used through out the dataset so that I can desing my regex accordingly.

In [10]:
if_website_or_contact_replace(df, "fees")
df["fees_clean"] = df["fees_clean"].apply(lambda x: "Not Available" if pd.isnull(x) else x)
symbols_list = extract_symbols(df)
symbols_list

['£', '$', '€']

Now, from all of the text in each row of the `fees` column, I will only keep the max amount and the currency name.

In [11]:
df["fees_clean"] = df["fees_clean"].apply(lambda x: extract_numeric_and_currency(x))
df.loc[:, ["fees","fees_clean"]]

,fees,fees_clean
0,Please see the university website for further information on fees for this course.,Not Available
1,Tuition fee per year (non-EU/EEA students): 15000 €,15000.0 EUR
2,"Home£13,470Overseas£27,230",27230.0 GBP
3,Please see the university website for further information on fees for this course.,Not Available
4,Please see the university website for further information on fees for this course.,Not Available
...,...,...
5995,Please see the university website for further information on fees for this course.,Not Available
5996,UK Fees: 2022/23 fees TBC*;2021/22 fees - 10400International Fees: 2022/23 fees TBC*;2021/22 fees - 17900,17900.0 EUR
5997,Please see the university website for further information on fees for this course.,Not Available
5998,Please see the university website for further information on fees for this course.,Not Available


Now, I will create two more columns and use these columns to make make a new column `FEE_EUR` where all the fees are in EUR.

In [12]:
# Apply extraction functions to create new columns
df['numeric_value'] = df['fees_clean'].apply(extract_numeric_value)
df['currency_name'] = df['fees_clean'].apply(extract_currency_name)

For the purpose of converting all other currencies to EUR, I will use v6.exchangerate-api.com as suggested by Chat GPT. I have already signed up and acquired my api key. 

In [13]:
my_api_key = "1b7f9ee2370ead6160f6bd45"
url = f"https://v6.exchangerate-api.com/v6/{my_api_key}/latest/EUR" # to access the url with EUR rates
response = requests.get(url) # to get the latest eur to other currencies rates
data = response.json() # storing the data in the variable data

# Apply the conversion using a lambda function with our pre-defined convert_to_eur function
df['FEE_EUR'] = df.apply(lambda row: convert_to_eur(row['numeric_value'], row['currency_name'], data), axis=1)

In [14]:
df['FEE_EUR'].info()

<class 'pandas.core.series.Series'>
Index: 5979 entries, 0 to 5999
Series name: FEE_EUR
Non-Null Count  Dtype  
--------------  -----  
1376 non-null   float64
dtypes: float64(1)
memory usage: 93.4 KB


Finally, I will delete the extra columns. Namely: `fees_clean`, `numeric_value`, `currency_name`.

In [15]:
df.columns
columns_to_drop = ["fees_clean", "numeric_value", "currency_name"]
df = df.drop(columns=columns_to_drop)

In [16]:
# Check if the data type of FEE_EUR column is float:
df['FEE_EUR'].dtype

dtype('float64')

### 2.1. Conjunctive query

#### 2.1.1 Create your index!
Before building the index,

- Create a file named `vocabulary`, in the format you prefer, that maps each word to an integer (`term_id`).

In [17]:
vocabulary = set() # set to remove duplicates and efficiency
# I will only used the description column as stated in the HW
df.description_clean.apply(lambda row: [vocabulary.add(word) for word in row])
vocabulary = list(vocabulary)

The variable vocabulary contains, in a list, all the words used in the description column. The question requires us to make a file named vocabulary that *maps each word to an integer (term_id).*

In [18]:
# maps each word to an integer (term_id)
vocab_with_index = dict()
unique_id = 0
for word in vocabulary:
  vocab_with_index[word] = unique_id
  unique_id+=1
vocab_with_index

with open("./vocabulary.json", "w") as file: # storing for reusability
    json.dump(vocab_with_index, file)

In [19]:
vocab_with_index = json.load(open("./vocabulary.json", "r")) # loading for use

Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

`{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}`

To create the inverted index, I can use the procedure used in the lab as follows:

In [20]:
Terms = pd.DataFrame(data=vocab_with_index.keys(), columns=['Term']); tqdm.pandas() # from lab
Terms['reverse'] = Terms.Term.progress_apply(lambda item: list(df.loc[df.description_clean.apply(lambda row: item in row)].index))

# word_and_appearances help with the search engine and inverted index. It contains as keys the words and as values a list which 
# represents the occurences of that word in our dataset

word_and_appearances = dict(zip(Terms.Term, Terms.reverse)) 

# The inverted index can also be made as commented below: 
# inverted_index = dict(zip(Terms.index, Terms['reverse'])); however I will implement my own function
Terms.head()

100%|██████████| 9704/9704 [01:34<00:00, 103.00it/s]


,Term,reverse
0,team,"[5, 65, 98, 104, 176, 220, 238, 274, 290, 348, 353, 371, 395, 396, 408, 592, 629, 661, 702, 775, 784, 785, 827, 829, 831, 844, 849, 850, 864, 870, 877, 938, 943, 993, 1006, 1019, 1031, 1046, 1140, 1152, 1168, 1173, 1184, 1187, 1191, 1194, 1234, 1301, 1314, 1335, 1464, 1509, 1531, 1539, 1618, 1620, 1638, 1942, 2022, 2024, 2042, 2064, 2082, 2145, 2157, 2162, 2169, 2173, 2212, 2235, 2326, 2399, 2461, 2501, 2537, 2538, 2561, 2562, 2573, 2592, 2622, 2624, 2628, 2637, 2640, 2705, 2769, 2773, 2777, 2791, 2835, 2943, 3006, 3014, 3036, 3074, 3107, 3161, 3162, 3163, ...]"
1,outstandingschool,[3413]
2,analysisdesign,[2127]
3,offer,"[1, 2, 10, 20, 21, 23, 33, 34, 36, 40, 41, 49, 52, 53, 58, 67, 69, 77, 78, 97, 119, 125, 128, 135, 151, 157, 158, 166, 186, 191, 201, 216, 231, 233, 237, 242, 247, 248, 249, 254, 259, 277, 285, 289, 293, 294, 296, 297, 298, 311, 318, 322, 331, 340, 357, 361, 384, 386, 399, 411, 437, 438, 443, 445, 466, 488, 499, 501, 504, 507, 545, 558, 562, 573, 584, 585, 587, 590, 591, 595, 607, 621, 624, 625, 630, 636, 638, 654, 655, 657, 668, 682, 686, 709, 715, 718, 730, 736, 737, 751, ...]"
4,gep,[3382]


Or, I can use the function that I created to make an inverted list. Here's how:

In [21]:
inverted_index = inverted_index(vocab_with_index, df)

I will now store this inverted index locally

In [22]:
with open("./inverted_index_1.json", "w") as file: # storing for reusability
    json.dump(inverted_index, file)
inverted_index = json.load(open("./inverted_index_1.json", "r")) # loading for use

### 2.1.2 Execute the query
Given a query input by the user, for example:

`advanced knowledge`

The Search Engine is supposed to return a list of documents.

---

For this task, I will first take the input from the user and then normalize the query using the same technique I used to preprocess the description column.

In [23]:
# user_input = input("Add your Input")
user_input = "advanced knowledge"
normalized_query = normalize_query(user_input)

Instead of using the inverted index which has the term_id and a list of where it appears in the document, I will use a related inverted index, which I created above with name word_and_appearances which contains words as keys and list of places they occur in the document as values.

In [24]:
new_dataset = search_engine(normalized_query, df, word_and_appearances).head()
new_dataset

,courseName,universityName,description,url
0,Allergy - MSc/PGDip/PGCert,Imperial College London,"Allergy is an increasing global health problem. There is an urgent need both in the UK and worldwide for training in clinical allergy.Our Allergy courses have been designed to suit all levels of healthcare professionals who deal with patients with allergic conditions, including doctors (GPs, Paediatricians, Specialists in Allergy, Dermatology, Respiratory medicine or ENT) as well as specialist nurses, dietitians, nutritionists and biomedical scientists.The course enables students to develop advanced knowledge and practical skills to optimally diagnose and manage allergic diseases at all ages.",www.findamasters.com/masters-degrees/course/allergy-msc-pgdip-pgcert/?i112d2578c11669
1,Digital Design and Branding MSc,Brunel University London,"Ask BrunelOur Digital Design and Branding MSc degree is a modern, hands-on course that combines digital technology and creative design. It has been designed to equip you with advanced knowledge and practical skills required for a creative role in the digital branding industry.You’ll develop deep understanding of advanced digital design practices within the context of branding.",www.findamasters.com/masters-degrees/course/digital-design-and-branding-msc/?i28d8297c28943
2,Digital Design and Manufacture MSc,University of Edinburgh,"Programme descriptionWith the transition to industry 4.0, digital design and manufacturing has become integral to society and is helping drive more efficient and less wasteful production processes.Our Digital Design and Manufacture programme will prepare you for a career in this rapidly changing industry by combining an advance knowledge of the digital technologies employed in advanced manufacture and key entrepreneurial skills.You will learn the fundamentals which underpin digital manufacturing; sensing, automation, information, additive manufacturing, simulation and modelling and will explore how these interact with data and machine learning.How will I learn?You will deepen your knowledge through:",www.findamasters.com/masters-degrees/course/digital-design-and-manufacture-msc/?i300d1312c66187
3,Analytical Sciences MSc,University of Bradford,"Our MSc in Analytical Sciences MSc is a research-focused degree.You’ll develop the experience and techniques required to be an independent practitioner of modern analytical science, with the skills required for research and by industry.It delivers core teaching in advanced aspects of analytical chemistry and enables you to explore areas of specialist knowledge through optional modules.You’ll apply a wide range of analytical techniques and gain invaluable research experience through an extended research project.Having developed the fundamentals of the analytical sciences, these core techniques are applied to disciplines including:",www.findamasters.com/masters-degrees/course/analytical-sciences-msc/?i285d7562c71587
4,International Business MSc,University of Leicester,This is for you if you want to enhance your existing skills to become a successful global business leader and gain advanced knowledge of issues facing international businesses today.Apply now for January 2024Submit course,www.findamasters.com/masters-degrees/course/international-business-msc/?i322d7917c72241


## 2.2 Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

Find all the documents that contain all the words in the query. Sort them by their similarity with the query.

Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

To solve this task, you must use the tfIdf score and the Cosine similarity. The field to consider is still the description.

#### 2.2.1 Make an Inverted index

Below, I will make an inverted index which contains as keys the unique term_id of the word, and as values, its corresponding occurences and tf-idf values in the dataset's row.

In [25]:
# Our inverted_index with tf-idf value as well.
inverted_index_tf_idf = inverted_index_tfidf(word_and_appearances, df)

In [26]:
with open("./inverted_index_tfidf.json", "w") as file: # storing for reusability
    json.dump(inverted_index, file)
inverted_index = json.load(open("./inverted_index_tfidf.json", "r")) # loading for use

### 2.2.2 Execute the query
In this new setting, given a query, you get the proper documents (i.e., those containing all the query's words) and sort them according to their similarity to the query. For this purpose, as the scoring function, we will use the Cosine Similarity concerning the tfIdf representations of the documents.

---

First, I will take the user input and normalize it.

In [27]:
# user_input = input("Add your Input")
user_input = "advanced knowledge"
normalized_query = normalize_query(user_input)

Time to compute and get the TF-IDF scores for our query:

In [28]:
query_tfidf = calculate_query_tfidf(normalized_query, word_and_appearances, df)
query_tfidf

{'advanc': 0.84, 'knowledg': 0.57}

I will now compute the intersection list of query and documents.

In [29]:
appearances = list()
[appearances.append(word_and_appearances[word]) for word in normalized_query]
# initialising the set with the value of the first list of appearances in the list of appearances of the word
intersection_list = set(appearances[0])
for appearance in appearances[1:]: # for the rest of the terms in the list of term ids
    intersection_list.intersection_update(appearance)
intersection_list = sorted(list(intersection_list))

I will not compute the similarity scores using cosine similarity with respect to the tf-idf scores. I will store these scores in a dictionary named as similarity_scores where each key is the document_id (row of the dataframe) and value is the cosine similarity of the document and query

In [30]:
similarity_scores = {}
for document_i in intersection_list:
    document_vector = df.at[document_i, "description_clean"] # getting the document vector at the row of interest
    # calculated the tf_idf of the document vector. It will result in words as keys and values as tf-idf scores
    document_tfidf = calculate_document_tfidf(document_vector, word_and_appearances, df) 
    # Compute cosine similarity
    dot_product = 0
    for word in normalized_query:
        if word in document_tfidf:
# computing the dot product of only the query's tfidf score and the tfidf score of that word in the document
            dot_product += query_tfidf[word] * document_tfidf[word] 

    norm_doc_i = np.linalg.norm(list(document_tfidf.values())) # computing norm of the doc vector
    norm_query = np.linalg.norm(list(query_tfidf.values()) )# computing norm of the query vector
    
    if norm_doc_i != 0 and norm_query != 0: # only the non zero results
        cosine_similarity_doc_i_query = dot_product / (norm_doc_i * norm_query)
        similarity_scores[document_i] = cosine_similarity_doc_i_query

Using heap data structure to retrieve top 10 similarity scores:

In [31]:
top_10_documents = heapq.nlargest(10, similarity_scores, key=similarity_scores.get)

Finally, bringing the result of the query search.

In [32]:
results = return_results(top_10_documents, similarity_scores, df)
results

,courseName,universityName,description,url,Cosine_Similarity
0,Advanced Healthcare Practice - MSc,Cardiff University,"Why study this courseOur MSc Advanced Healthcare Practice programme aims to develop your knowledge, understanding and critical appreciation of the four pillars of advanced level practice.It offers you the opportunity to apply learning to advance your leadership and management, facilitation of learning and teaching, and clinical practice skills, all of which are underpinned by evidence, research and service improvement.Our programme is suitable for those progressing towards a level of autonomous advanced practice as well as experienced registered health care professionals, already working as advanced practitioners.",www.findamasters.com/masters-degrees/course/advanced-healthcare-practice-msc/?i33d4736c67062,0.354660
1,Advanced Clinical Practice MSc,University of Greenwich,Learn essential strategies and prepare for leadership roles in advanced clinical practice with this engaging Master’s in Advanced Clinical Practice.Develop your skills and deepen your knowledge of advanced health clinical practice with this tailored Master's course. Our MSc in Advanced Clinical Practice is designed for current practitioners who are registered with a professional body and who would like to become advanced clinical practitioners.,www.findamasters.com/masters-degrees/course/advanced-clinical-practice-msc/?i309d6650c56313,0.353597
2,Advanced Computing MSc,King’s College London,"Our Advanced Computing MSc provides knowledge and experience of computing at an advanced level. The programme allows students to select modules on a wide range of advanced computer science subjects, so that they can build a programme that suits their interests and career aspirations.",www.findamasters.com/masters-degrees/course/advanced-computing-msc/?i132d3905c23524,0.347863
3,Advancing Practice - MSc,University of Northampton,"Our MSc Advancing Practice awards support the advancement of healthcare professionals practice by developing, knowledge, skills and understanding to challenge and innovate future practice. This award also offers and alternative route to individuals with an interest in Advanced Clinical Practice.By challenging legal, professional and ethical dilemmas you will develop advanced knowledge and skills to underpin safe and effective practice, supported by the application of complex decision making. You will benefit from a flexible route which builds and develops opportunity for advancing healthcare provision.",www.findamasters.com/masters-degrees/course/advancing-practice-msc/?i337d1774c57759,0.337683
4,Advanced Mechanical Engineering - MSc (Eng),University of Leeds,"This course offers a broad range of advanced subjects to develop your knowledge across the mechanical engineering disciplines. If you’re a graduate engineer who wishes to pursue a career in industry using advanced engineering techniques or want to gain in-depth knowledge for a career in research or academia, this course will help you do",www.findamasters.com/masters-degrees/course/advanced-mechanical-engineering-msc-eng/?i321d1002c12327,0.290262
5,Advanced Clinical Practice - MSc,Canterbury Christ Church University,"Gain the knowledge and skills needed to become a qualified health care professional working in a clinically senior post.The MSc Advanced Clinical Practice course has been developed to provide you with knowledge and skills to allow you to develop and progress your advanced clinical practice role. You will develop an in-depth and advanced knowledge of your role (ACP). The knowledge is informed by current practice and research. You will develop a critical awareness of the subject matter and be able to demonstrate critical skills, knowledge of your profession demonstrating strategic leadership and education in practice, you will also reflect on your progress as a learner.",www.findamasters.com/masters-degrees/course/advanced-clinical-practice-msc/?i32d2712c57002,0.28163

# 3. Define a new score!

Now it's your turn: build a new metric to rank MSc degrees.

Practically:

- The user will enter a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 2.1.
- Once you have the documents, you need to sort them according to your new score. In this step, you won't have any more to take into account just the description field of the documents; you can use also the remaining variables in your dataset (or new possible variables that you can create from the existing ones or scrape again from the original web-pages). You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

N.B.: You have to define a scoring function, not a filter!

The output, must contain:

- `courseName`
- `universityName`
- `description`
- `URL`
- The new `similarity score` of the documents with respect to the query

Are the results you obtain better than with the previous scoring function? Explain and compare results.

---

The idea behind the development of the new scoring function is an extension of the concept introduced in *Q2.2* to encompass other variables. This involves assigning varying weights based on the input query. This approach is motivated by the understanding that a user may have diverse interests when searching for information, such as specifying a city, a university name, or the course name. In such cases, if the query aligns with one of these specific variables, it is deemed to be more significant than others. This allows for a more nuanced and personalized scoring system tailored to the user's preferences and priorities.

Initially, we developed a function to efficiently compute weights based on whether the query matches any of the variables. Emphasis is given to the variable that matches at least a word of the input query, assigning higher importance to that specific variable.

After that we use the 'fuzzywuzzy' package, importing Levenshtein Distance, to calculate the similarity between the query and the variable value for each one of them.

In [33]:
user_query = "data science" 
normalized_query = normalize_query(user_query)
term_ids = get_term_id_from_query(vocab_with_index, normalized_query)
new_dataset = search_engine_full(term_ids, df, inverted_index)
# Let's start from the smaller_dataset derived from the engine.py
user_query_2 = "data science lancaster" # input user query

weights = calculate_weights(user_query_2, new_dataset) # computes the weights taking in input a second more specific query

# add the new_score as a new column of the df
new_dataset['score'] = new_dataset.apply(lambda row: calculate_total_score(user_query_2, row, weights), axis=1)

heap = []
for index, row in new_dataset.iterrows():
    heapq.heappush(heap, (-row['score'], index))

# Extract results
sorted_indices = [heapq.heappop(heap)[1] for _ in range(len(heap))]

# Create new df sorted
sorted_df = new_dataset.loc[sorted_indices]
k = 5 # k-top elements sorted by the new score
sorted_df[['courseName', 'universityName', 'description', 'score']].head(k)


,courseName,universityName,description,score
330,Data Science MSc,Lancaster University,"From business and finance to health and medicine, from infrastructure to societal studies, data science plays a vital role in all aspects of the modern world. Our MSc programme will ensure you have an advanced level of skills, knowledge, and experience in this rapidly expanding, highly in-demand field to achieve your career aspirations.",0.428696
267,Data Science - MSc,Lancaster University,"From business and finance to health and medicine, from infrastructure to societal studies, data science plays a vital role in all aspects of the modern world. Our MSc programme will ensure you have an advanced level of skills, knowledge, and experience in this rapidly expanding, highly in-demand field to achieve your career aspirations.",0.411304
211,Health Data Science MSc,Lancaster University,"Deep statistical thinking combined with expertise in health and computer science is becoming increasingly fundamental in tackling public health problems across the world. The MSc in Health Data Science will equip you with advanced technical skills which will allow you to develop a career as a data-scientist in the health and care sector.The MSc in Health Data Science, consists of an initial set of 4 core modules: “Statistical methods and models for health research”, “Programming for Health Data Science”, “Fundamentals for Health Data Science” and “Introduction to applied epidemiology”. These will allow you to develop and consolidate foundational skills in the three main areas of Health Data Science: epidemiology, statistics and computer science.",0.401739
210,Health Data Science MSc,Lancaster University,"Deep statistical thinking combined with expertise in health and computer science is becoming increasingly fundamental in tackling public health problems across the world. The MSc in Health Data Science will equip you with advanced technical skills which will allow you to develop a career as a data-scientist in the health and care sector.The MSc in Health Data Science, consists of an initial set of 4 core modules: “Statistical methods and models for health research”, “Programming for Health Data Science”, “Fundamentals for Health Data Science” and “Introduction to applied epidemiology”. These will allow you to develop and consolidate foundational skills in the three main areas of Health Data Science: epidemiology, statistics and computer science.",0.397391
327,Data Science MSc,Northumbria University,"This exciting Data Science master's has been designed by top academics in the field, in close consultation with leading data scientists from the industry and the Northumbria University Institute of Coding (IoC). It will provide you with the relevant skills needed to analyse, synthesise and manage different types and sizes of data efficiently.This programme is designed to train and produce data scientists who will fill a range of jobs requiring skills in methodical and statistical data analysis and help organisations (e.g., businesses, healthcare providers, financial institutions, industries) make the most of their huge amounts of data. You will develop knowledge insight from a variety of structured and unstructured data, using a range of data analysis methods, processes, algorithms, and systems.",0.392174


After several attempts, it can be concluded that the new similarity score yields more robust results and is more versatile compared to cosine similarity. This improvement stems from the consideration of additional variables, along with the analysis of the user query to gain insights into the user's search intent. This assumption is grounded in the idea that the user formulates queries with the intention of obtaining the most relevant results from the database, prioritizing them for easier visualization.

As we can also see in the output above, focusing solely on the '*description*' variable (as in Q2.2) can lead to biased results that may not align with the input query. Conversely, the new scoring approach demonstrates a more meaningful and relevant outcome.

Moreover, the utilization of Levenshtein distance as a string metric proves to be highly beneficial for quantifying the dissimilarity between two sequences, especially when they exhibit similarities. Informally, the Levenshtein distance between two words represents the minimum number of single-character edits (insertions, deletions, or substitutions) required to transform one word into the other. We opted for this measure precisely because, quite often, the input may contain errors such as missing or extra letters or concatenated words. The Levenshtein measure exhibits remarkable robustness in accommodating these variations.

# 4. Visualizing the most relevant MSc degrees


Using maps can help people understand how far one university is from another so they can plan their academic careers more adequately. Here, we challenge you to show a map of the courses found with the score defined in point 3. You should be able to identify at least the city and country for each MSc degree. You can find some ideas on how to create maps in Python [here](https://plotly.com/python/maps/) and [here](https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621) but you will maybe need further information for a proper visualization, like coordinates (latitude and longitude). You can retrieve this data using various tools:

1. [Here](https://medium.com/@manilwagle/geocoding-the-world-using-google-api-and-python-1f6b6fb6ca48) you can find a helpful tutorial on how to encode geo-informations using Google API in Python (this tool can also be used in Google Sheets)
2. You can collect a list of unique places in the format (City, Country) and ask chatGPT (or, as usual, any other LLM chatbot) to provide you with a list of corresponding representative coordinates
3. Explore and find the best solution for your case!
Once you defined your visualization strategy, include a way to encode fees in your charts. The map should show (with a proper legend) different courses and associated taxation: the user wants a glimpse not only of how far he will need to move but also of how much it will cost him!

---

Let's start by creating a new column with the full address (`universityName`, `city`, `country`), starting from the dataframe obtained in point 3. So this will be easier for google API to understand the address. It also increases the accuracy of geo-coding.

In [34]:
k = 100
sorted_df['full_address'] = sorted_df['universityName'] + ", " + sorted_df['city'] + ", " + sorted_df['country']
sorted_df.head(k)

,courseName,universityName,facultyName,city,country,description,FEE_EUR,url,score,full_address
330,Data Science MSc,Lancaster University,Department of Mathematics and Statistics,Lancaster,United Kingdom,"From business and finance to health and medicine, from infrastructure to societal studies, data science plays a vital role in all aspects of the modern world. Our MSc programme will ensure you have an advanced level of skills, knowledge, and experience in this rapidly expanding, highly in-demand field to achieve your career aspirations.",NaN,www.findamasters.com/masters-degrees/course/data-science-msc/?i136d2395c69561,0.428696,"Lancaster University, Lancaster, United Kingdom"
267,Data Science - MSc,Lancaster University,School of Computing and Communications,Lancaster,United Kingdom,"From business and finance to health and medicine, from infrastructure to societal studies, data science plays a vital role in all aspects of the modern world. Our MSc programme will ensure you have an advanced level of skills, knowledge, and experience in this rapidly expanding, highly in-demand field to achieve your career aspirations.",NaN,www.findamasters.com/masters-degrees/course/data-science-msc/?i136d2396c50699,0.411304,"Lancaster University, Lancaster, United Kingdom"
211,Health Data Science MSc,Lancaster University,Lancaster Medical School,Lancaster,United Kingdom,"Deep statistical thinking combined with expertise in health and computer science is becoming increasingly fundamental in tackling public health problems across the world. The MSc in Health Data Science will equip you with advanced technical skills which will allow you to develop a career as a data-scientist in the health and care sector.The MSc in Health Data Science, consists of an initial set of 4 core modules: “Statistical methods and models for health research”, “Programming for Health Data Science”, “Fundamentals for Health Data Science” and “Introduction to applied epidemiology”. These will allow you to develop and consolidate foundational skills in the three main areas of Health Data Science: epidemiology, statistics and computer science.",NaN,www.findamasters.com/masters-degrees/course/health-data-science-msc/?i136d6518c65318,0.401739,"Lancaster University, Lancaster, United Kingdom"
210,Health Data Science MSc,Lancaster University,Division of Health Research,Lancaster,United Kingdom,"Deep statistical thinking combined with expertise in health and computer science is becoming increasingly fundamental in tackling public health problems across the world. The MSc in Health Data Science will equip you with advanced technical skills which will allow you to develop a career as a data-scientist in the health and care sector.The MSc in Health Data Science, consists of an initial set of 4 core modules: “Statistical methods and models for health research”, “Programming for Health Data Science”, “Fundamentals for Health Data Science” and “Introduction to applied epidemiology”. These will allow you to develop and consolidate foundational skills in the three main areas of Health Data Science: epidemiology, statistics and computer science.",NaN,www.findamasters.com/masters-degrees/course/health-data-science-msc/?i136d2424c69573,0.397391,"Lancaster University, Lancaster, United Kingdom"
327,Data Science MSc,Northumbria University,Computer and Information Sciences,Newcastle,United Kingdom,"This exciting Data Science master's has been designed by top academics in the field, in close consultation with leading data scientists from the industry and the Northumbria University Institute of Coding (IoC). It will provide you with the relevant skills needed to analyse, synthesise and manage different types and sizes of data efficiently.This programme is designed to train and produce data scientists who will fill a range of jobs requiring skills in methodical and statistical data analysis and help organisations (e.g., businesses, healthcare providers, financial institutions, industries) make the 

All the libraries in the first part are required for geo coding where as Kelper and geopandas are required for mapping. Then we need to obtain from Google the API key to start geocoding. For safety reasons it will not be displayed in this code.

In [35]:
# Load the required libraries
import pandas as pd
# from pandas_profiling import ProfileReport
from googlemaps import Client as GoogleMaps
import googlemaps
import gmaps
from keplergl import KeplerGl
import geopandas as gpd

gmaps = googlemaps.Client(key='') # API key obtained from Google (not displayed for safety reasons)

In [36]:
addresses1= sorted_df.iloc[:,-1:]
addresses1.head(10)

,full_address
330,"Lancaster University, Lancaster, United Kingdom"
267,"Lancaster University, Lancaster, United Kingdom"
211,"Lancaster University, Lancaster, United Kingdom"
210,"Lancaster University, Lancaster, United Kingdom"
327,"Northumbria University, Newcastle, United Kingdom"
329,"University of Chester, Chester, United Kingdom"
321,"Wageningen University & Research, Wageningen, Netherlands"
302,"University of Westminster, London, United Kingdom"
255,"Politecnico di Milano, Milan, Italy"
328,"University of Exeter, Exeter, United Kingdom"


Here, I have created two empty columns called `long` and `lat` to store the coordinates that I will get back after running the addresses against the API Key.

In [37]:
addresses1['long'] = ""
addresses1['lat'] = ""

In [38]:
for x in range(len(addresses1)):
    geocode_result = gmaps.geocode(addresses1['full_address'][x])
    addresses1['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
    addresses1['long'][x] = geocode_result[0]['geometry']['location']['lng']

In [39]:
addresses1

,full_address,long,lat
330,"Lancaster University, Lancaster, United Kingdom",-2.787729,54.010394
267,"Lancaster University, Lancaster, United Kingdom",-2.787729,54.010394
211,"Lancaster University, Lancaster, United Kingdom",-2.787729,54.010394
210,"Lancaster University, Lancaster, United Kingdom",-2.787729,54.010394
327,"Northumbria University, Newcastle, United Kingdom",-1.61778,54.978252
...,...,...,...
295,"University College London, London, United Kingdom",-0.13404,51.524559
74,"University College Cork, Cork, Ireland",-8.491198,51.893609
63,"University of Groningen, Groningen, Netherlands",6.56283,53.219327
107,"WHU, Koblenz, Germany",7.613507,50.400325


To avoid having the same points on the map, that represent the same university, exactly overlapped and indistinguishable, let's introduce a small random factor to slightly shift the coordinates without significantly altering the position.

In [40]:
addresses1['lat'] += np.random.normal(-0.005, 0.005, len(addresses1))
addresses1['long'] += np.random.normal(-0.005, 0.005, len(addresses1))
addresses1

,full_address,long,lat
330,"Lancaster University, Lancaster, United Kingdom",-2.787235,54.0078
267,"Lancaster University, Lancaster, United Kingdom",-2.798491,54.007137
211,"Lancaster University, Lancaster, United Kingdom",-2.795717,54.004665
210,"Lancaster University, Lancaster, United Kingdom",-2.792362,53.997091
327,"Northumbria University, Newcastle, United Kingdom",-1.627439,54.981972
...,...,...,...
295,"University College London, London, United Kingdom",-0.13377,51.519018
74,"University College Cork, Cork, Ireland",-8.501857,51.887814
63,"University of Groningen, Groningen, Netherlands",6.554877,53.21268
107,"WHU, Koblenz, Germany",7.618646,50.389903


In [41]:
# Lets join the results with original file
sorted_df['Lat']=addresses1['lat']
sorted_df['Lon']= addresses1['long']
sorted_df.head()

,courseName,universityName,facultyName,city,country,description,FEE_EUR,url,score,full_address,Lat,Lon
330,Data Science MSc,Lancaster University,Department of Mathematics and Statistics,Lancaster,United Kingdom,"From business and finance to health and medicine, from infrastructure to societal studies, data science plays a vital role in all aspects of the modern world. Our MSc programme will ensure you have an advanced level of skills, knowledge, and experience in this rapidly expanding, highly in-demand field to achieve your career aspirations.",NaN,www.findamasters.com/masters-degrees/course/data-science-msc/?i136d2395c69561,0.428696,"Lancaster University, Lancaster, United Kingdom",54.0078,-2.787235
267,Data Science - MSc,Lancaster University,School of Computing and Communications,Lancaster,United Kingdom,"From business and finance to health and medicine, from infrastructure to societal studies, data science plays a vital role in all aspects of the modern world. Our MSc programme will ensure you have an advanced level of skills, knowledge, and experience in this rapidly expanding, highly in-demand field to achieve your career aspirations.",NaN,www.findamasters.com/masters-degrees/course/data-science-msc/?i136d2396c50699,0.411304,"Lancaster University, Lancaster, United Kingdom",54.007137,-2.798491
211,Health Data Science MSc,Lancaster University,Lancaster Medical School,Lancaster,United Kingdom,"Deep statistical thinking combined with expertise in health and computer science is becoming increasingly fundamental in tackling public health problems across the world. The MSc in Health Data Science will equip you with advanced technical skills which will allow you to develop a career as a data-scientist in the health and care sector.The MSc in Health Data Science, consists of an initial set of 4 core modules: “Statistical methods and models for health research”, “Programming for Health Data Science”, “Fundamentals for Health Data Science” and “Introduction to applied epidemiology”. These will allow you to develop and consolidate foundational skills in the three main areas of Health Data Science: epidemiology, statistics and computer science.",NaN,www.findamasters.com/masters-degrees/course/health-data-science-msc/?i136d6518c65318,0.401739,"Lancaster University, Lancaster, United Kingdom",54.004665,-2.795717
210,Health Data Science MSc,Lancaster University,Division of Health Research,Lancaster,United Kingdom,"Deep statistical thinking combined with expertise in health and computer science is becoming increasingly fundamental in tackling public health problems across the world. The MSc in Health Data Science will equip you with advanced technical skills which will allow you to develop a career as a data-scientist in the health and care sector.The MSc in Health Data Science, consists of an initial set of 4 core modules: “Statistical methods and models for health research”, “Programming for Health Data Science”, “Fundamentals for Health Data Science” and “Introduction to applied epidemiology”. These will allow you to develop and consolidate foundational skills in the three main areas of Health Data Science: epidemiology, statistics and computer science.",NaN,www.findamasters.com/masters-degrees/course/health-data-science-msc/?i136d2424c69573,0.397391,"Lancaster University, Lancaster, United Kingdom",53.997091,-2.792362
327,Data Science MSc,Northumbria University,Computer and Information Sciences,Newcastle,United Kingdom,"This exciting Data Science master's has been designed by top academics in the field, in close consultation with leading data scientists from the industry and the Northumbria University Institute of Coding (IoC). It will provide you with the relevant skills needed to analyse, synthesise and manage different types and sizes of data efficiently.This programme is designed to train and produce data scientists who will fill a range of jobs requiring skills in methodical and statistical data analysis and help organisations 

Now, in order to display the `FEE_EUR` column on the map, we apply some changes to that.

In [42]:
sorted_df['FEE_EUR'][sorted_df['FEE_EUR'].isnull()] = np.NaN
sorted_df['FEE_EUR'] = sorted_df['FEE_EUR'].astype(float)
sorted_df['FEE_EUR'].dropna().apply(round, 3)

14      1000
337       18
336    18079
358    14311
105    35500
       ...  
166    19222
66     52384
201    20728
74     18500
107    37200
Name: FEE_EUR, Length: 95, dtype: int64

Now we divide the FEE_EUR column into classes, for a quicker and more compact visualization of prices on the map.

This approach allows for a clearer representation, making it easier to discern patterns and variations in pricing. By categorizing the prices into classes, we create a visual summary that simplifies the interpretation of data points. This can enhance the overall understanding of the spatial distribution of prices and facilitate comparisons between different regions or locations on the map.

In [74]:
arr = sorted_df['FEE_EUR'].values
bins = [np.nanmin(arr), np.nanpercentile(arr, 20), np.nanpercentile(arr, 40), np.nanpercentile(arr, 60), np.nanpercentile(arr, 80), np.nanmax(arr), np.nan]
labels = [f"{round(bins[0], 2)}-{round(bins[1], 2)}", f"{round(bins[1], 2)}-{round(bins[2], 2)}", f"{round(bins[2], 2)}-{round(bins[3], 2)}", f"{round(bins[3], 2)}-{round(bins[4], 2)}", f"{round(bins[4], 2)}-{round(bins[5], 2)}", "nan"]

sorted_df['FEE_EUR_binned'] = pd.cut(arr, bins=bins, labels=labels)


In [75]:
sorted_df['FEE_EUR_binned']

330                   NaN
267                   NaN
211                   NaN
210                   NaN
327                   NaN
              ...        
295                   NaN
74      16236.76-20057.58
63                    NaN
107    30343.16-209610.98
25                    NaN
Name: FEE_EUR_binned, Length: 362, dtype: category
Categories (6, object): ['17.85-13149.24' < '13149.24-16236.76' < '16236.76-20057.58' < '20057.58-30343.16' < '30343.16-209610.98' < 'nan']

Now it's time to create the map.

In [76]:
# Create a basemap 
maps = KeplerGl(height=600, width=800) # show the map
# Create a gepdataframe
gdf = gpd.GeoDataFrame(sorted_df, geometry=gpd.points_from_xy(sorted_df.Lon, sorted_df.Lat)) # Add data to Kepler
map_2 = KeplerGl(height=400, data={"data_1": gdf})
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':                                                                                   cou…

At the conclusion, we save the map data into an HTML file. To enhance the information, we incorporate the FEE_EUR_binned column for each university. This allows us to visualize, using distinct colors, the respective fees associated with specific courses for those seeking admission. Moreover, by selecting a particular university on the map, users can view the precise fees associated with that course.

The colors vary in intensity based on the price range, ranging from dark red, indicating a lower price range, to yellow, indicating a higher cost.

**NB**: the yellow color shows that the price is not indicated in the df (nan value).

In [78]:
map_2.save_to_html(file_name="map_courses.html")

Map saved to map_courses.html!


# 5. BONUS: More complex search engine

IMPORTANT: This is a bonus step, so it's not mandatory. You can get the maximum score also without doing this. We will take this into account, only if the rest of the homework has been completed.

For the Bonus part, we want to ask you more sophisticated search engine. Here we want to let users issue more complex queries. The options of this new search engine are:

1. Give the possibility to specify queries for the following features (the user should have the option to issue none or all of them):
- `courseName`
- `universityName`
- `universityCity`
2. Specify a range for the fees to retrieve only MSc whose taxation is in that range.

3. Specify a list of countries which the search engine should only return the courses taking place in city within those countries.
4. Filter based on the courses that have already started.
5. Filter based on the presence of online modality.
Note 1: You should be aware that you should give the user the possibility to select any of the abovementioned options. How should the user use the options? We will accept any manual that you provide to the user.

Note 2: As you may have realized from 1st option, you need to build inverted indexes for those values and return all of the documents that have the similarity more than 0 concerning the given queries. Choose a logical way to aggregate the similarity coming from each of them and explain your idea in detail.

Note 3: The options other than 1st one can be considered as filtering criteria so the retrieved documents must respect all of those filters.

The output must contain the following information about the places:

- `courseName`
- `universityName`
- `url`

---

# 7. Algorithmic Question

Leonardo is an intern at a company. He is paid based on the total number of hours he has worked. They agreed d days ago that Leonardo could not work less than `l_i` or more than `r_i` hours per i-th day. Furthermore, he was warned by HR that on his last day at the company, he should provide a detailed report on how many hours he worked each day for the previous `d` days.

Today is the day Leonardo should report to HR, but the problem is that he didn't account for how many hours he put in for each day, so he only has the total sum of the hours (`total_hours`) he put in total in these `d` days. He believes that if he creates a report in which each number `x_i` corresponds to the total hours he worked on the i-th day while satisfying the HR limitations and the total sum of all `x_i` equals `total_hours`, he would be fine.

He cannot create such a report independently and requests your assistance. He will give you the number of days `d`, total hours spent `total_hours`, and the HR limitations for each day (`l_i`, `r_i`), and he wants you to assist him in determining whether it is possible to create such a fake report. If that is possible, make such a report.

**Input**

The first line of input contains two integers `d` and `total_hours` - the number of days Leonardo worked there and the total number of hours he worked for the company. Each of the following `d` lines contains two integer numbers `l_i` and `r_i` - the minimum and maximum hours he can work on the i-th day.

**Output**

If such a report cannot be generated, print 'NO' in one output line. If such a report is possible, print 'YES' in the output and `d` numbers - the number of hours Leonardo spent each day - in the second line. If more than one solution exists, print any of them.


### 1. Implement a code to solve the above mentioned problem.

In [ ]:
d, total_hours = map(int, input().split())
limits = []
for _ in range(d):
    min_hours, max_hours = map(int, input().split())
    limits.append((min_hours, max_hours))

hours = [0]*d
for i in range(d):
    if total_hours < limits[i][0]:
        print('NO')
    hours[i] = limits[i][0]
    total_hours -= limits[i][0]

if total_hours >= 0:
    for i in range(d):
        if hours[i] < limits[i][1]:
            hours[i] += 1
            total_hours -= 1
    if total_hours == 0:
        print('YES')
        print(*hours)

### 2. What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.

Overall time complexity (the Big O notation) of the solution: $O(1) + O(d)+ O(d) + O(d)+ O(d)= O(d)$

Checking the individual time complexities by breaking down the operations:

In [ ]:
d, total_hours = map(int, input().split())
# Reading the input is a constant time operation and has a time complexity of O(1).
limits = []
for _ in range(d):
    min_hours, max_hours = map(int, input().split())
    limits.append((min_hours, max_hours))

The next part of code creates a list and appends it with $d$ pairs of integers.
It involves a loop that runs d times, so it has a time complexity of $O(d)$.

In [ ]:
hours = [0]*d
# This operation creates a list of d lenght and initializes each element to zero, so it has a time complexity of O(d).

for i in range(d):
    if total_hours < limits[i][0]:
        print('NO')
    hours[i] = limits[i][0]
    total_hours -= limits[i][0]

This next loop iterates over a range of $d$ days. Each iteration involves a constant number of operations and the time complexity is $O(d)$.

In [ ]:
if total_hours >= 0:
    for i in range(d):
        if hours[i] < limits[i][1]:
            hours[i] += 1
            total_hours -= 1
    if total_hours == 0:
        print('YES')
        print(*hours)

Like the one from before, this loop iterates over a range of $d$ days. Each iteration involves constant time operations and the time complexity is also $O(d)$.

### 3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the two differ, which one is right? (why?)

ChatGPT had a different breakdown of the solutions time complexity, but the calcuations and result is the same: $O(d)$.

- Reading input: The first line reads two integers, which takes constant time. The loop that follows reads d pairs of integers, which has a time complexity of $O(d)$.
- Initializing the hours list: This operation has a time complexity of $O(d)$ since it involves creating a list of length d and initializing each element to zero.
- First loop (for i in range(d)): This loop iterates over the range of days (d). Each iteration involves a constant number of operations (assignments and condition checks) and does not depend on the size of the input. Therefore, the time complexity of this loop is0 $O(d)$.
- Second loop (for i in range(d)): Similar to the first loop, it iterates over the range of days (d). Each iteration involves constant time operations. The time complexity of this loop is also $O(d)$.
So, the overall time complexity of the provided code is $O(d)$, where d is the number of days.

### 4. What do you think of the optimality of your code? Do you believe it is optimal? Can you improve? Please elaborate on your response.

Given the task, I believe the solution code to be optimal. However, improvements could be made regarding readability and efficiency. The assignment of minimum hours and additional hours could be combined into the same loops. Another improvement could consist of using list comprehension when creating the limits list.